In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 
from tensorflow.keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasClassifier
import pickle

In [2]:
data=pd.read_csv('Churn_Modelling.csv')
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

one_hot_encoder_geo=OneHotEncoder(handle_unknown='ignore')
geo_encoded = one_hot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))

data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
X = data.drop('Exited',axis=1)
y = data['Exited']

X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=42)

scaler= StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [3]:
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('one_hot_encoder_geo.pkl','wb') as file:
    pickle.dump(one_hot_encoder_geo,file)

with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)  
    

In [11]:
def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],))),
    
    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))
    
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])
        
    return model

In [12]:
model=KerasClassifier(build_fn=create_model,layers=1,neurons=32,verbose=1)

In [14]:
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}

In [15]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3,verbose=1)
grid_result = grid.fit(X_train, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits

Epoch 1/50


c:\Development\Project3\venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)




250/250 [==============================] - 1s 1ms/step - loss: 0.4495 - accuracy: 0.8081
Epoch 2/50
250/250 [==============================] - 0s 1ms/step - loss: 0.3952 - accuracy: 0.8342
Epoch 3/50
250/250 [==============================] - 0s 1ms/step - loss: 0.3690 - accuracy: 0.8474
Epoch 4/50
250/250 [==============================] - 0s 1ms/step - loss: 0.3555 - accuracy: 0.8553
Epoch 5/50
250/250 [==============================] - 0s 1ms/step - loss: 0.3493 - accuracy: 0.8561
Epoch 6/50
250/250 [==============================] - 0s 1ms/step - loss: 0.3457 - accuracy: 0.8601
Epoch 7/50
250/250 [==============================] - 0s 1ms/step - loss: 0.3425 - accuracy: 0.8555
Epoch 8/50
250/250 [==============================] - 0s 1ms/step - loss: 0.3410 - accuracy: 0.8596
Epoch 9/50
250/250 [==============================] - 0s 1ms/step - loss: 0.3385 - accuracy: 0.8600
Epoch 10/50
250/250 [==============================] - 0s 1ms/step - loss: 0.3380 - accuracy: 0.8611
Epoch 11

In [16]:
print("Best : %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best : 0.856750 using {'epochs': 50, 'layers': 1, 'neurons': 128}
